# **LoopsResultsAnalysis**

## **1. Importing and First Proccesing**

In [72]:
import numpy as np
import pandas as pd
from scipy import stats
from DataCleaning import *
from ProcessingConfig import *

pd.options.display.max_columns = 50

In [73]:
raw_data = pd.read_excel(cleaning_config['raw_data_path'])
print(f'original shape: {raw_data.shape}')
raw_data = clean_data(raw_data)
print(f'final shape: {raw_data.shape}')

original shape: (13294, 32)
-- drop_first_loop: 1630 rows were filtered out.
-- drop_first_line: 2437 rows were filtered out.
-- filter_trail_outliers: 721 rows were filtered out.
-- filter_step_outliers: 242 rows were filtered out.
final shape: (8264, 26)


In [74]:
n_subjects = raw_data['subject'].nunique()
print(f'number of subjects: {n_subjects}')

n_sessions = raw_data[['subject', 'trial_set']].drop_duplicates().shape[0]
print(f'number of sessions: {n_sessions}')

number of subjects: 15
number of sessions: 29


In [75]:
# enveloping t_test and f_test functions

def f_test(smp1, smp2, alpha):
    f_score = smp1.std() / smp2.std()
    df1, df2 = smp1.size - 1, smp2.size - 1
    return stats.f.cdf(f_score, df1, df2) >= alpha

def t_test_ind(smp1, smp2, alpha, alternative='two-sided'):
    equal_var = f_test(smp1, smp2, alpha)
    t_score, p_value = stats.ttest_ind(smp1, smp2, alternative=alternative, equal_var=equal_var)
    
    print(f'p_value: {p_value}')
    if p_value <= alpha:
        print('There is a difference between the samples!')
    else:
        print('It is not possible to determine whether there is an effect.')
    return t_score, p_value

## **2. Priming Effect Testing**

In [81]:
alpha = 0.05

In [76]:
switching_diff = pd.DataFrame()

# mean response time and success rate, group by loop type and loop switching.
switching_diff['mean_response_time'] = raw_data[raw_data['correct']].groupby(['loop_type', 'loop_type_switch'])['rt'].mean()
switching_diff['mean_success_rate'] = raw_data.groupby(['loop_type', 'loop_type_switch'])['correct'].mean()

# loop ending response time, group by loop type and loop switching.
loop_end_mask = (raw_data['response_needed']) & (raw_data['expected_response'].isnull()) & (raw_data['correct'])
switching_diff['loop_end_mean_response_time'] = raw_data.loc[loop_end_mask].groupby(['loop_type', 'loop_type_switch'])['rt'].mean()

switching_diff

mean_response_time  mean_success_rate  \
loop_type loop_type_switch                                          
for       False                    1304.817951           0.977205   
          True                     1339.163618           0.981546   
while     False                    1567.088924           0.978626   
          True                     1574.004692           0.975585   

                            loop_end_mean_response_time  
loop_type loop_type_switch                               
for       False                              591.438721  
          True                               583.686941  
while     False                              813.898551  
          True                               788.949550

In [77]:
# checking for priming effect on 'for' loops
same = raw_data.loc[raw_data['correct'] & ( ~ raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'for'), 'rt']
different = raw_data.loc[raw_data['correct'] & (raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'for'), 'rt']

t_score, p_value = t_test_ind(different, same, alpha=alpha, alternative='greater')

p_value: 0.18413260693230632
It is not possible to determine whether there is an effect.


In [78]:
# checking for priming effect on 'while' loops
same = raw_data.loc[raw_data['correct'] & ( ~ raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'while'), 'rt']
different = raw_data.loc[raw_data['correct'] & (raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'while'), 'rt']

t_score, p_value = t_test_ind(different, same, alpha=alpha, alternative='greater')

p_value: 0.43659566007509937
It is not possible to determine whether there is an effect.


## **3. Difference Between Loop Types**

In [79]:
# general difference between response time in 'while' loops and 'for' loops
for_rt = raw_data.loc[raw_data['correct'] & (raw_data['loop_type'] == 'for'), 'rt']
while_rt = raw_data.loc[raw_data['correct'] & (raw_data['loop_type'] == 'while'), 'rt']

t_score, p_value = t_test_ind(while_rt, for_rt, alpha=alpha, alternative='greater')

p_value: 3.915993895607329e-18
There is a difference between the samples!


In [80]:
# difference between 'end loop' step response time in 'while' loops and 'for' loops
for_end_loop_rt = raw_data.loc[loop_end_mask & (raw_data['loop_type'] == 'for'), 'rt']
while_end_loop_rt = raw_data.loc[loop_end_mask & (raw_data['loop_type'] == 'while'), 'rt']

t_score, p_value = t_test_ind(while_end_loop_rt, for_end_loop_rt, alpha=alpha, alternative='greater')

p_value: 2.5893518354792905e-11
There is a difference between the samples!
